In [1]:
# the output directory for generator inference results on the test set
# traits = ['o', 'c', 'e', 'a', 'n']
traits = ['c']
in_dirs = {}
for trait in traits:
    in_dirs[trait] = f'/data/user_data/wenkail/llm_personality/generator/data/outputs/generated_predictions_{trait}.jsonl'
in_dirs

{'c': '/data/user_data/wenkail/llm_personality/generator/data/outputs/generated_predictions_c.jsonl'}

In [2]:
import json
import re
def convert_data(in_file, trait):
    """in_file: innput file path, a jsonl file
    """
    data = [json.loads(i) for i in open(in_file, 'r').readlines()]
    pattern = r"Help me complete the sentence with certain Big Five Personality: (?:Openness|Conscientiousness|Extraversion|Agreeableness|Neuroticism) - (?:low|high)\n"
    
    data_lst = []
    for item in data:
        # get label and input, output as a csv file
        # label: ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
        # input: message
        message = item['prompt'].split("assistant\n\n")
        if len(message) == 1:
            continue
        message = re.split(pattern, message[0])[-1] + " " + item['predict'].split("\n\n")[0]
        
        info = {
            "ope_z_label": 2,
            "con_z_label": 2,
            "ext_z_label": 2,
            "agr_z_label": 2,
            "neu_z_label": 2,
            "message": message
        }
        
        label = 2
        if "- high" in item['prompt']:
            label = 1
        else:
            label = 0
        
        if trait == 'o':
            info['ope_z_label'] = label
        elif trait == 'c':
            info['con_z_label'] = label
        elif trait == 'e':
            info['ext_z_label'] = label
        elif trait == 'a':
            info['agr_z_label'] = label
        elif trait == 'n':
            info['neu_z_label'] = label
            
        data_lst.append(info)
    return data_lst

In [3]:
import pandas as pd
for trait in traits:
    res = convert_data(in_dirs[trait], trait)
    df = pd.DataFrame().from_records(res)
    df.to_csv(f"/data/user_data/wenkail/llm_personality/generator/data/outputs/generator_predictions_{trait}.csv")

In [ ]:
# from utils import preprocess_function_with_tokenizer
# for split in ['train', 'val', 'test']:
#     df = pd.read_csv(f'/data/user_data/wenkail/llm_personality/data/big5_data_classifier_{split}.csv')
#     dataset = Dataset.from_pandas(df.dropna())
#     tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
#     dataset = dataset.map(lambda examples: preprocess_function_with_tokenizer(examples, tokenizer), batched=True)
#     dataset.save_to_disk(f'/data/user_data/wenkail/llm_personality/data_mse/{split}_psychgen')